# Embedding Layer vs. Linear Layer

- PyTorch의 embedding layer는 linear layer 처럼 동일한 MM(Matrix Multiplication)을 수행함.
- 어떤 차이가 있는지를 확인.

In [1]:
import torch

print("PyTorch version:", torch.__version__)

PyTorch version: 2.4.1


## Embedding layer (nn.Embedding)

In [2]:
# 간단한 예시를 통한 확인
idx = torch.tensor([2, 3, 1])
num_idx = max(idx)+1

# hyperparameter. 원하는 출력 shape를 지정할 수 있음.
out_dim = 5

In [6]:
torch.manual_seed(62)

embedding = torch.nn.Embedding(num_idx, out_dim)

# 현재 Embedding layer의 shape 및 weight의 확인이 가능
embedding, embedding.weight, embedding.weight.shape

(Embedding(4, 5),
 Parameter containing:
 tensor([[-0.8964,  1.6584, -0.6848, -0.9976,  0.3654],
         [ 0.9772,  1.1116, -2.3113,  0.0635, -1.9439],
         [-0.2049,  0.8689, -0.8124,  0.7084,  0.6774],
         [-1.1003,  0.1116, -0.5711,  0.6550, -0.2539]], requires_grad=True),
 torch.Size([4, 5]))

- ID가 1인 data의 dense vector representation을 다음처럼 얻을 수 있음.
  - 즉, 2번째(idx(=1)+1)의 embedding vector

In [7]:
embedding(torch.tensor([1]))

tensor([[ 0.9772,  1.1116, -2.3113,  0.0635, -1.9439]],
       grad_fn=<EmbeddingBackward0>)

In [8]:
# 앞서 생성한 예시 idx tensor를 embedding layer에 전달하면, 해당 index에 해당하는 weight들이 추출됨.
idx = torch.tensor([2, 3, 1])
embedding(idx)

tensor([[-0.2049,  0.8689, -0.8124,  0.7084,  0.6774],
        [-1.1003,  0.1116, -0.5711,  0.6550, -0.2539],
        [ 0.9772,  1.1116, -2.3113,  0.0635, -1.9439]],
       grad_fn=<EmbeddingBackward0>)

- 이미지를 통해 확인하면 다음과 같음:
![embedding_visualization](./images/tensor_embedding_example.png)

## Linear layer (nn.Linear)

- 앞서 생성한 embedding layer가 PyTorch에서 one-hot encoding에 대한 nn.Linear와 정확히 도출한다는 것을 확인.

In [9]:
onehot = torch.nn.functional.one_hot(idx)
onehot

tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]])

In [10]:
torch.manual_seed(62)
linear = torch.nn.Linear(num_idx, out_dim, bias=False)

linear, linear.weight, linear.weight.shape

(Linear(in_features=4, out_features=5, bias=False),
 Parameter containing:
 tensor([[ 0.1412,  0.3290, -0.2885,  0.0220],
         [ 0.0978,  0.1350, -0.1811,  0.0358],
         [ 0.1423,  0.4220, -0.0185, -0.0967],
         [ 0.4569,  0.2168, -0.0856,  0.4267],
         [ 0.0107,  0.4084, -0.4149,  0.4295]], requires_grad=True),
 torch.Size([5, 4]))

- PyTorch의 nn.Linear는 random weight로 초기화(random initialization) 되므로, 동일한 random weight를 사용해야 함.
- 따라서, 가중치를 다시 재할당.

In [12]:
linear.weight = torch.nn.Parameter(embedding.weight.T)
linear(onehot.float()), embedding(idx)

(tensor([[-0.2049,  0.8689, -0.8124,  0.7084,  0.6774],
         [-1.1003,  0.1116, -0.5711,  0.6550, -0.2539],
         [ 0.9772,  1.1116, -2.3113,  0.0635, -1.9439]], grad_fn=<MmBackward0>),
 tensor([[-0.2049,  0.8689, -0.8124,  0.7084,  0.6774],
         [-1.1003,  0.1116, -0.5711,  0.6550, -0.2539],
         [ 0.9772,  1.1116, -2.3113,  0.0635, -1.9439]],
        grad_fn=<EmbeddingBackward0>))

- 볼 수 있듯이, embedding layer를 거쳐서 나온 값과 완전히 일치함.
- 이를 도표로 확인하면 다음과 같음.
![example](./images/embedding_matmul_visualization.png)